In [19]:
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
dataPath = 'FisherIris.csv'
attributeNames = []

# Dump data file into an array
with open(dataPath, "r") as ins:
    listArray = []
    for line in ins:
        # Remove junk, irritating formating stuff
        listArray.append(line.replace('\n', '').split('\t'))

In [3]:
# Encode data in desired format
n = len(listArray) - 1
p = len(listArray[0][0].split(',')) - 1
X = np.zeros((n, p))
y = np.zeros(n)
for i, data in enumerate(listArray):
    dataTemp = data[0].split(',')
    if i == 0: # first row is attribute names
        attributeNames = dataTemp[0:4]
    else:
        X[i - 1,:] = dataTemp[0:4]
        flowerInd = dataTemp[4]
        if flowerInd == 'Setosa':
            y[i-1] = 0
        elif flowerInd == "Versicolor":
            y[i-1] = 1
        else:
            y[i-1] = 2

In [7]:
# Actual Fisher discriminant done after here
# Looping over pi, mu and S, since calculations are similar for all classes
pi = np.zeros(3)
mu = np.zeros((3, p))
S = np.zeros((p,p))
#S = 0
for i in range(3):
    nk = len(np.where(y == i)[0])
    XSubset = X[np.where(y == i)[0], :]
    pi[i]  = nk/n
    mu[i,:] = np.sum(XSubset/nk, axis=0)

    p1 = XSubset-mu[i,:];
    p2 = np.dot(np.transpose(p1),p1)
#    p3 = np.divide(p2,(n-i))
    
    S += p2
    
    # means and a transposed XSubset

    # And remember to scale with number of DF, check result with size of S
# S[0,0] should be something like 0.265
S = S / (n-3)

print S

[[ 0.26500816  0.09308163  0.16744218  0.03847347]
 [ 0.09308163  0.11588435  0.0552381   0.03342313]
 [ 0.16744218  0.0552381   0.18517007  0.0425415 ]
 [ 0.03847347  0.03342313  0.0425415   0.04201088]]


In [40]:
# Discriminants


def produceDiscriminantLine(X, S, mu, pi):
    Sinv = np.linalg.inv(S) # How to inverse S
    Xtra = np.transpose(X)
    print mu
    # First "part" of the calc, something like X * inv(S) * mu' ...
    first = Xtra*Sinv*mu # * mu
    # Find the "second" part in notes
    mutra = np.transpose(mu)
    second = (mutra * Sinv * mutra) /2
    third = np.log(pi)
    return first - second + third

d = np.zeros((3, n))
for i in range(3): # We are lazy programmers, therefor we loop
    d[i,:] = produceDiscriminantLine(X, S, mu[i,:], pi[i])

[ 5.006  3.418  1.464  0.244]


ValueError: operands could not be broadcast together with shapes (4,150) (4,4) 

In [ ]:
# Classify according to discriminant
yhat = np.unravel_index(np.argmax(d, axis=0), d.shape)[1] # index on "1" to get indexes
confusion_matrix(y, yhat) # can also do manually by just checking if y == yhat and counting